In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df["label"] = (df.points >= 90).astype(int)
df = df[["country","description", "label"]]

In [ ]:
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=1024):
  df = dataframe.copy()
  labels = df.pop('label')
  df = df["country","description"]
  ds = tf.data.Dataset.from_tensor_slices((df, labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

In [ ]:
train_data = df_to_dataset(train)
valid_data = df_to_dataset(val)
test_data = df_to_dataset(test)

# LSTM

In [ ]:
encoder = tf.keras.layers.TextVectorization(max_tokens=2000)
encoder.adapt(train_data.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'and', 'the', 'a', 'of', 'with', 'this', 'is', 'wine',
       'flavors', 'in', 'it', 'to', 'its', 'on', 'fruit', 'aromas',
       'palate', 'that'], dtype='<U17')

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=32,
        mask_zero=True
    ),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.evaluate(train_data)
model.evaluate(valid_data)

13/13 [==============================] - 2s 186ms/step - loss: 0.6927 - accuracy: 0.5360


[0.6927294731140137, 0.5360468029975891]

In [ ]:
history = model.fit(train_data, epochs=5, validation_data=valid_data)

Epoch 1/5
102/102 [==============================] - 96s 503ms/step - loss: 0.5570 - accuracy: 0.7030 - val_loss: 0.3759 - val_accuracy: 0.8258
Epoch 2/5
102/102 [==============================] - 52s 506ms/step - loss: 0.3704 - accuracy: 0.8342 - val_loss: 0.3480 - val_accuracy: 0.8387
Epoch 3/5
102/102 [==============================] - 52s 511ms/step - loss: 0.3501 - accuracy: 0.8445 - val_loss: 0.3502 - val_accuracy: 0.8374
Epoch 4/5
102/102 [==============================] - 57s 553ms/step - loss: 0.3426 - accuracy: 0.8468 - val_loss: 0.3453 - val_accuracy: 0.8396
Epoch 5/5
102/102 [==============================] - 52s 507ms/step - loss: 0.3386 - accuracy: 0.8486 - val_loss: 0.3424 - val_accuracy: 0.8415


In [ ]:
model.evaluate(test_data)

13/13 [==============================] - 4s 273ms/step - loss: 0.3497 - accuracy: 0.8416


[0.3497326672077179, 0.8415910005569458]